In [2]:
import random
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
MIN_PER_RANK = 1  # Nodes/Rank: How 'fat' the DAG should be.
MAX_PER_RANK = 5
MIN_RANKS = 3     # Ranks: How 'tall' the DAG should be.
MAX_RANKS = 5
PERCENT = 30      # Chance of having an Edge.

def generate_dag(min_w, max_w, total_nodes): # min_w, max_w: node values range, total_nodes: total number of nodes
    random.seed()  # Initialize the random number generator

    G = nx.DiGraph()

    current_nodes = 0 # Total number of nodes in the graph
    ranks = [] # Number of nodes in each rank

    # Generate ranks with nodes until the total number of nodes is reached
    while current_nodes < total_nodes:
        new_nodes = min(MAX_PER_RANK, total_nodes - current_nodes) # Number of nodes in the new rank
        ranks.append(new_nodes) # Add the new rank to the list of ranks
        current_nodes += new_nodes # Update the total number of nodes

    nodes = 1 # Total number of nodes in the graph starts from 1

    for rank in ranks:
        for k in range(rank):
            # Assign a random weight to each new node
            node_weight = random.randint(min_w, max_w)
            G.add_node(nodes + k, weight=node_weight)

        # Edges from old nodes ('nodes') to new ones ('rank').
        for j in range(nodes - 1): # Adjusted to start from 0
            for k in range(rank):
                if random.randint(0, 99) < PERCENT: # Randomly decide if there is an edge between the nodes
                    G.add_edge(j + 1, k + nodes) # Adjusted to start from 1

        nodes += rank  # Accumulate into old node set.

    # remove isolated nodes
    G.remove_nodes_from(list(nx.isolates(G)))

    root_id = 0 # Root node is 0

    roots = [node for node in G.nodes() if G.in_degree(node) == 0] # Find the root nodes
    for _ in roots:
        G.add_edge(root_id, _) # Add an edge from the root node to each root node

    # add an attribute 'out' to the root node as a set containing 0
    G.nodes[root_id]['out'] = set()
    G.nodes[root_id]['out'].add(0)
    G.nodes[root_id]['weight'] = 0

    return G

In [4]:
def compute_node_out(G, node_id):
    node_weight = G.nodes[node_id]['weight']
    predecessors = list(G.predecessors(node_id))

    node_out = set()
    for predecessor in predecessors:
        predecessor_out = G.nodes[predecessor]['out']
        node_out = node_out.union(predecessor_out)

    node_out = set([x + node_weight for x in node_out])
    return node_out


def rank(G, node_id):
    rank =[]
    for element in G.nodes[node_id]['out']:
        rank.append((element - G.nodes[node_id]['weight'] + 1, element))

    rank = sorted(rank, key=lambda x: x[0])

    return rank


def process_graph(G):
    for node in list(nx.topological_sort(G))[1::]: # the root is already initialized with (0)
        G.nodes[node]['out'] = compute_node_out(G, node)

    for node in G.nodes():
        G.nodes[node]['optimal_father'] = None

In [8]:
# random generated DAG
G = generate_dag(1, 10, 30) #(min weight, max weight, total number of nodes)
process_graph(G)

for node in G.nodes():
    print(f"\nNode {node} | weight: {G.nodes[node]['weight']} \nout: {sorted(list(G.nodes[node]['out']))}")


Node 1 | weight: 4 
out: [4]

Node 2 | weight: 9 
out: [9]

Node 3 | weight: 1 
out: [1]

Node 4 | weight: 8 
out: [8]

Node 5 | weight: 6 
out: [6]

Node 6 | weight: 7 
out: [11]

Node 7 | weight: 1 
out: [5, 7, 10]

Node 8 | weight: 1 
out: [5, 7, 9]

Node 9 | weight: 8 
out: [16]

Node 10 | weight: 10 
out: [11, 14]

Node 11 | weight: 1 
out: [7, 9]

Node 12 | weight: 3 
out: [7, 8, 9, 10, 12, 13, 14, 17, 19]

Node 13 | weight: 8 
out: [8]

Node 14 | weight: 1 
out: [2, 6, 8, 10, 12]

Node 15 | weight: 10 
out: [26]

Node 16 | weight: 8 
out: [10, 14, 15, 16, 17, 18, 20, 34]

Node 17 | weight: 2 
out: [4, 8, 10, 12, 13, 14, 16, 18]

Node 18 | weight: 7 
out: [12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 26]

Node 19 | weight: 4 
out: [15]

Node 20 | weight: 1 
out: [3, 6, 7, 8, 9, 10, 11, 12, 13, 15]

Node 21 | weight: 6 
out: [9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 40]

Node 22 | weight: 8 
out: [11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]

Node 23 | weight: 2 
out:

# Compressione con Elias-Fano

### Compressione di un nodo espresso in maniera diretta

Abbiamo una lista di interi strettamente crescenti: eseguiamo i seguenti step per comprimere la lista

* **shift left**: sottraiamo ad ogni elemento il valore del primo elemento della lista
    ```python
    for i in range(1, n):
        a[i] -= a[0]
    ```
    In questo modo il primo elemento sarà sempre 0.

    > Costo: $O(n)$

* **subtract rank:** sottraiamo ad ogni elemento (0-index) il numero di elementi minori di esso (che essendo strettamente crescente, equivale a sottraire l'indice)
    ```python
    for i in range(n):
        a[i] -= i
    ```
    Questo possiamo farlo perché Elias-Fano funziona anche su sequenze di interi non decrescenti

    > Costo: $O(n)$

    
* **flag bit**: dobbiamo salvare il primo elemento della lista originale per poter fare la decompressione. Se usiamo un encoding per intero come il $\gamma$-encoding, non occupiamo troppo spazio. 

* **encoding**: Elias-Fano encoding per la lista di interi

    > Costo: $O(n)$ compressione e decompressione

* **RRR**: usiamo una struttura RRR per poter fare il rank e il select su bit array

    > Costo: $o(n)$ in spazio e successivamente $O(1)$ per rank e select

Per liste di grosse dimensioni e sopratutto per liste di interi molto grandi, la compressione sarà molto efficiente.

## Rappresentazione e compressione dei nodi in maniera indiretta rispetto ai successori

Noi sappiamo che il set `out` associato ad ogni nodo è costruito prendendo l'unione dei set dei suoi predecessori ed aggiungendo il valore del nodo stesso ad ogni elemento

--- 

La compressione però va fatta in funzione di uno dei successori del nodo (se presenti). Sappiamo che il set `out` del nodo sarà un sottoinsieme del set `out` del suo successore meno il suo valore. Quindi quello che possiamo fare per un nodo che vogliamo storare implicitamente è non storare in maniera esplicita il set `out` del nodo, ma solo i seguenti valori:

* **node_weight (int)**: intero che rappresenta il valore del nodo
* **successor_id (int)**: un riferimento al nodo padre  (ce ne saranno sicuramente di più, dobbiamo scegliere quello ottimale)
* **offset (list of int)**: la posizione (0-index) dei valori del set `out` del nodo padre che sono presenti nel set `out` del nodo corrente

In questo se vogliamo accedere al set `out` del nodo corrente, possiamo fare un `access` al set `out` del nodo padre e prendere gli elementi a partire dalla posizione `offset` fino alla fine. Con RRR possiamo fare il rank e il select per trovare la posizione di un elemento nel set `out` del nodo padre in $O(1)$.

**OSSERVAZIONI:** Andando avanti nel dag, storare il set `out` esplicitamente diventa molto costoso in quanto potrebbero salire parecchio i valori facendo tutte le addizioni. In questo modo ad ogni nodo il massimo numero che può capitare è pari alla lunghezza del set `out` del nodo padre.

## Dubbi

* Come faccio a decidere quale è il padre ottimale? A questo punto è quello con la lunghezza minore del set `out`? 
* Come decido quali nodi comprimere in maniera diretta e quali in maniera indiretta? 
  * Facendo le cose a livelli di _topological generations_ a questo punto non ha senso (?) perché in quel caso potrei non storare nulla oltre il valore del nodo. E se mi viene chiesto il set out di un nodo implicito faccio la set union di tutti i predecessori e aggiungo il suo valore. Questo implicherebbe salvare i livelli uno si e uno no.
  
  > A grossi questa cosa non piace. Piuttosto propone questa sotto

  * Come detto sopra fare la compressione di un nodo rispetto al suo successore. Il problema sta nel capire quali sono i nodi in cui storare esplicitamente il set 'out' e quali no. Si deve in qualche modo evitare - o trovare una soluzione - al caso in cui abbiamo un nodo implicito ed il suo padre ottimale è pure implicito e così via. Non vorrei dover andare troppo avanti nel grafo prima di trovare un nodo esplicito.
    * Più che altro: se un nodo lo storo implicitamente, questo potrebbe essere l'optimal father di qualcun altro. 
  
### Si potrebbe iniziare trovando delle euristiche per capire quali nodi comprimere in maniera diretta e quali in maniera indiretta.

????????
Quali euristiche prendo? Mi vengono in mente cose a livelli, ma livello per livello a sto punto faccio il ragionamento della set union dei precedenti ed ignoro il resto